This is the implementation of example V.B. of the paper *Variational approach to rare event simulation using least-squares regression, C. Hartmann, O. Kebiri, L. Neureither, L. Richter,
submitted to Chaos, 2019.*

We consider the Ornstein-Uhlenbeck SDE
$$d X_t = (\mu-X_t ) dt + \sigma d B_t$$
and want to compute the expectation value
$$\mathbb{E}[\exp(-\alpha X_T) | X_0 = x]$$
by approximating the corresponding optimal control in an importance sampling attempt.

In [ ]:
%matplotlib inline

import copy
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import time
import torch as pt 

from numpy import exp, sqrt
from scipy.linalg import inv

## Shooting method with neural networks

In [ ]:
class NN(pt.nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.nn_dims = [1, 20, 1]
        self.W = [item for sublist in 
                  [[pt.nn.Parameter(pt.randn(self.nn_dims[i], self.nn_dims[i + 1], requires_grad=True)),
                    pt.nn.Parameter(pt.randn(self.nn_dims[i + 1], requires_grad=True))] for
                   i in range(len(self.nn_dims) - 1)]
                  for item in sublist]
        for i, w in enumerate(self.W):
            self.register_parameter('param %d' % i, w)
        
        self.BN1 = pt.nn.BatchNorm1d(self.nn_dims[0])
        self.BN2 = pt.nn.BatchNorm1d(self.nn_dims[1])
        self.BN3 = pt.nn.BatchNorm1d(self.nn_dims[2])
        self.BN = [self.BN1, self.BN2, self.BN3]
        
        self.adam = pt.optim.Adam(self.parameters(), lr=0.01)
        
    def forward(self, x):
        x = self.BN[0](x)
        for i in range(len(self.nn_dims) - 1):
            x = pt.matmul(x, self.W[2 * i]) # + self.W[2 * i + 1]
            x = self.BN[i + 1](x)
            if i != len(self.nn_dims) - 2:
                x = pt.nn.functional.relu(x)
        return x
    
class Y_0(pt.nn.Module):
    def __init__(self):
        super(Y_0, self).__init__()
        self.Y_0 = pt.nn.Parameter(pt.randn(1), requires_grad=True)
        self.register_parameter('param', self.Y_0)
        self.adam = pt.optim.Adam(self.parameters(), lr=0.01)
        
    def forward(self, x):
        return pt.ones(x.shape[0]) * self.Y_0

In [ ]:
u = 1

def u_t(u, t, T):
    return -sigma.item() * u * exp(t - T)

def gamma_t(u, t, T, x):
    return u * x * exp(t - T) - u**2 * sigma**2 / 4 * (1 - exp(2 * (t - T)))

def g(x):
    return u * x

In [ ]:
T = 5
sigma = pt.sqrt(pt.tensor(2.0))

delta_t = pt.tensor(0.05)    # time discretization of SDE
sq_delta_t = pt.sqrt(delta_t)
N = int(T / delta_t)

K = 50    # batch size
L = 10000    # gradient steps

nn_y = Y_0()
nn_z = [NN() for i in range(N)]

sd_nn_y = copy.deepcopy(nn_y.state_dict())
sd_nn_z = [copy.deepcopy(nn.state_dict()) for nn in nn_z]

nn_y.load_state_dict(sd_nn_y)
for nn, sd_nn in zip(nn_z, sd_nn_z):
    nn.load_state_dict(sd_nn)

nn_y.train()
for nn in nn_z:
    nn.train()

loss_log = []
Y_0_log = []
times = []

for l in range(L):
    t_0 = time.time()
    X = pt.zeros([K, N + 1])
    Y = pt.zeros([K, N + 1])
    X[:, 0] = pt.zeros(K)
    Y[:, 0] = nn_y.forward(X[:, 0])

    xi = pt.randn(K, N + 1)
    for i in range(N):
        Z = -nn_z[i].forward(X[:, i].unsqueeze(1)).squeeze(1)
        # c = 0
        c = -Z
        # c = u_t(u, i * delta_t, T)
        X[:, i+1] = X[:, i] + (- X[:, i] + sigma * c) * delta_t + sigma * sq_delta_t * xi[:, i+1] # - sigma * Z
        Y[:, i+1] = Y[:, i] + delta_t * (0.5 * Z.pow(2) + c * Z) + sq_delta_t * Z * xi[:, i+1] # - 0.5
    
    nn_y.adam.zero_grad()
    for nn in nn_z:
        nn.adam.zero_grad()

    loss = (Y[:, N] - g(X[:, N])).pow(2).mean()
    loss.backward()

    nn_y.adam.step()
    for nn in nn_z:
        nn.adam.step()   

    loss_log.append(pt.mean((Y[:, N] - g(X[:, N]))**2).item())
    
    t_1 = time.time()
    times.append(t_1 - t_0)
    Y_0_log.append(Y[0, 0].item())
    
    if (l % 100 == 0):
        print('%d - loss: %.4e - Y_0: %.4e - time per iteration: %.2fs' % (l, loss_log[-1], Y[0, 0].item(), np.mean(times[-100:])))
        
#loss_log_NN_naive = loss_log.copy()
loss_log_NN_control = loss_log.copy()

## Ansatz functions

In [ ]:
M_gaussian = 5
mus = np.linspace(-2, 2, M_gaussian)
sigma_basis = 1

def phi_m_gaussian(x, m):
    return 1 / sqrt(2 * np.pi * sigma_basis**2) * exp(- (x - mus[m])**2 / (2 * sigma_basis**2))

def phi_constant(x):
    return np.ones(len(x))

def phi(x, ansatz):
    if ansatz == 'gaussian':
        return [phi_m_gaussian(x, m) for m in range(M_gaussian)]
    if ansatz == 'constant':
        return [phi_constant(x)]

def grad_phi_m(x, m):
    return -(x - mus[m]) / (sqrt(2 * np.pi) * sigma_basis**3) * exp(- (x - mus[m])**2 / (2 * sigma_basis**2))

def grad_phi_gaussian(x):
    return [grad_phi_m(x, m) for m in range(M_gaussian)]

def gamma(x, n):
    return alpha[n, :].dot(phi(x))

def grad_gamma(x, n, alpha, ansatz):
    return alpha[n, :].dot(phi(x, ansatz))

def grad_gamma_LSMC_gaussian(x, n, alpha):
    return alpha[n, :].dot(grad_phi_gaussian(x))

In [ ]:
x_val = np.linspace(-4, 4, 400);

for m in range(M_gaussian):
    plt.plot(x_val, phi_m_gaussian(x_val, m))

## Shooting method with ansatz functions

In [ ]:
ansatz = 'gaussian'
M = 5    # amount of ansatz functions
T = 5    # time horizon
eta = 0.05    # learning rate
sigma = sqrt(2.0)    # noise of SDE

delta_t = 0.05    # time discretization of SDE
N = int((T / delta_t) + 1)
K = 100    # batch size
L = 1000000    # gradient steps

X = np.zeros([K, N + 1], dtype=np.float32)
Y = np.zeros([K, N + 1], dtype=np.float32)
Z = np.zeros([K, N + 1], dtype=np.float32)
alpha = np.zeros([N + 1, M], dtype=np.float32)
alpha_Y = 0

loss_log = []

for l in range(L):
    X[:, 0] = np.zeros(K)    # np.random.uniform(K)
    Y[:, 0] = np.ones(K) * alpha_Y
    xi = np.random.randn(K, N + 1)
    for i in range(N):
        X[:, i+1] = X[:, i] + (- X[:, i]) * delta_t + sigma * sqrt(delta_t) * xi[:, i+1]
        Y[:, i+1] = (Y[:, i] + delta_t * 0.5 * (sigma * grad_gamma(X[:, i], i, alpha, ansatz))**2
                     + sqrt(delta_t) * sigma * grad_gamma(X[:, i], i, alpha, ansatz) * xi[:, i+1])
    
    # compute gradients and update
    for i in range(N):
        for m in range(M):
            alpha[i, m] -= eta * np.mean(2 * (Y[:, N] - g(X[:, N])) * (delta_t * sigma**2 * grad_gamma(X[:, i], i, alpha, ansatz) * phi(X[:, i], ansatz)[m] + sqrt(delta_t) * sigma * phi(X[:, i], ansatz)[m] * xi[:, i+1]))
    
    alpha_Y -= eta * np.mean(2 * (Y[:, N] - g(X[:, N])))
    
    loss_log.append(np.mean((Y[:, N] - g(X[:, N]))**2))
        
    if (l % 100 == 0):
        print("%d - loss: %.4e" % (l, loss_log[l]))
        
#alpha_constant = alpha.copy()
#loss_log_ansatz = loss_log.copy()
alpha_gaussian = alpha.copy()
loss_log_gaussian = loss_log.copy()

## LSMC with ansatz functions

In [ ]:
def phi_m_linear(x, m):
    if m == 0:
        return np.ones(len(x))
    if m == 1:
        return x
    
def grad_phi_m_linear(x, m):
    if m == 0:
        return np.zeros(len(x))
    if m == 1:
        return np.ones(len(x))
    
def grad_phi(x):
    return [grad_phi_m_linear(x, m) for m in range(2)]

def grad_gamma_LSMC_linear(x, n, alpha):
    return alpha[n, :].dot(grad_phi(x))

In [ ]:
M = M_gaussian    # amount of ansatz functions
#M = 2
T = 5    # time horizon
K = 10000    # samples
delta_t = 0.05    # time discretization of SDE
N = int(T / delta_t)

X = np.zeros([K, N + 1])
Y = np.zeros([K, N + 1])
Z = np.zeros([K, N + 1])
alpha = np.zeros([N + 1, M])

X[:, 0] = 0.0 * np.ones(K)

for i in range(N):
    xi = np.random.randn(K)
    X[:, i+1] = X[:, i] + (- X[:, i]) * delta_t + sigma * sqrt(delta_t) * xi

Y[:, N] = g(X[:, N])
Z[:, N] = sigma * u * np.ones(K)


for n in range(N-1, 0, -1):

    A_n = np.zeros([K, M])
    for m in range(M):
        A_n[:, m] = phi_m_gaussian(X[:, n], m)
        # A_n[:, m] = phi_m_linear(X[:, n], m)
        
    b_n = Y[:, n+1] - delta_t * 0.5 * Z[:, n+1]**2
    alpha[n, :] = inv(A_n.T.dot(A_n)).dot(A_n.T).dot(b_n)
    Y[:, n] = A_n.dot(alpha[n, :])
    
    A_n = np.zeros([K, M])
    for m in range(M):
        A_n[:, m] = grad_phi_m(X[:, n], m)
        # A_n[:, m] = grad_phi_m_linear(X[:, n], m)
    
    Z[:, n] = sigma * A_n.dot(alpha[n, :])
    
alpha_LSMC_gaussian = copy.deepcopy(alpha)
#alpha_LSMC_linear = copy.deepcopy(alpha)

## Comparison of attempts

In [ ]:
nn_y.eval()
for nn in nn_z:
    nn.eval()
    
x = -1.0    # plotted x value
t = 3    # plotted t value
n = int(np.ceil(t / delta_t))

plt.rcParams.update({'font.size': 13})

fig, ax = plt.subplots(2, 1, figsize=(6, 8)) 
t_range = np.linspace(0, T, N + 1)
n_range = range(0, N + 1)
x_range = np.linspace(-3, 3, 100)

ax[0].set_title('optimal control for x = %.0f' % x)
ax[0].plot(t_range[1:-1], - sigma * grad_gamma_LSMC_linear([x], n_range[1:-1], alpha_LSMC_linear), '.', ms=8, markevery=4, label='LSMC, constant ansatz', color='blue')
ax[0].plot(t_range[1:-1], - sigma * grad_gamma_LSMC_gaussian([x], n_range[1:-1], alpha_LSMC_gaussian), '--', linewidth=1, label='LSMC, Gaussian ansatz', color='blue')
ax[0].plot(t_range[2:], [nn_z[n].forward((pt.tensor([[x]]))).item() for n in n_range[1:-1]], 'x', ms=10, mew=2, markevery=4, label='shooting, NN', color='green');
ax[0].plot(t_range, - sigma * grad_gamma([x], np.array(n_range), alpha_constant[:-1], 'constant'), '.', ms=8, markevery=4, label='shooting, constant ansatz', color='red')
ax[0].plot(t_range, - sigma * grad_gamma([x], np.array(n_range), alpha_gaussian[:-1], 'gaussian'), '--', linewidth=1, ms=7, markevery=4, label='shooting, Gaussian ansatz', color='red')
ax[0].plot(t_range, u_t(u, t_range, T), color='grey', label=r'true solution $u^*(x, t)$');
ax[0].set_xlabel('t');
ax[0].set_ylabel(r'$u^*(x, t)$');
ax[0].legend();

ax[1].set_title('optimal control for t = %.0f' % t)
ax[1].plot(x_range, - sigma * grad_gamma_LSMC_linear(x_range, n + 1, alpha_LSMC_linear), '.', ms=8, markevery=5, label='LSMC, constant ansatz', color='blue')
ax[1].plot(x_range, - sigma * grad_gamma_LSMC_gaussian(x_range, n + 1, alpha_LSMC_gaussian), '--', linewidth=1, label='LSMC, Gaussian ansatz', color='blue')
ax[1].plot(x_range, [nn_z[n].forward((pt.tensor([[x]]))).item() for x in x_range], 'x', ms=10, mew=2, markevery=4, label='shooting, NN', color='green');
ax[1].plot(x_range, - sigma * grad_gamma(x_range, n + 1, alpha_constant, 'constant'), '.', ms=8, markevery=6, label='sgooting, constant ansatz', color='red');
ax[1].plot(x_range, - sigma * grad_gamma(x_range, n + 1, alpha_gaussian, 'gaussian'), '--', linewidth=1, ms=7, markevery=4, label='shooting, Gaussian ansatz', color='red');
ax[1].plot(x_range, np.ones(len(x_range)) * u_t(u, t, T), color='grey', label=r'true solution $u^*(x, t)$');
#ax[1].legend()
ax[1].set_xlabel('x');
ax[1].set_ylabel(r'$u^*(x, t)$');
ax[1].set_ylim(-0.3, 0);
fig.tight_layout()
#fig.savefig('img/OU_approximation_comparison.pdf')

In [ ]:
plt.rcParams.update({'font.size': 14})

start = 0
end = 1700
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
ax.plot(loss_log_naive[start:end], label='NN, uncontrolled')
ax.plot(loss_log_NN_control[start:end], label='NN, controlled', color='grey')
ax.plot(loss_log_ansatz[start:end], label='constant ansatz', color='orange')
ax.plot(loss_log_gaussian[start:end], label='Gaussian ansatz', color='green')
ax.set_yscale('log')
ax.set_xlabel('gradient steps');
ax.set_ylabel('loss');
ax.legend();
#fig.savefig('img/OU_loss_comparison.pdf');